# Explorative Data Analysis

In [1]:
import pandas as pd 
import numpy as np

%matplotlib inline

import matplotlib as mpl 
import matplotlib.pyplot as plt

import seaborn as sns

import plotly
import plotly.graph_objects as go 

In [2]:
mpl.rcParams['figure.figsize'] = (20,16)
pd.set_option('display.max_rows', 500)

sns.set(style = 'darkgrid')

In [3]:
df_plot = pd.read_csv('../data/processed/COVID_small_flattable.csv',sep = ',')
df_plot.head()


,date;Italy;US;India;Brazil
0,2020-01-22;0;1;0;0
1,2020-01-23;0;1;0;0
2,2020-01-24;0;2;0;0
3,2020-01-25;0;2;0;0
4,2020-01-26;0;5;0;0


In [4]:
# Importing the data for the world population

wp = pd.read_csv('../data/vaccination/COVID-19-master/data_tables/world_pop_by_country.csv')

wp.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/vaccination/COVID-19-master/data_tables/world_pop_by_country.csv'

# Matplotlib example

In [ ]:
plt.figure()

ax = df_plot.iloc[20:,:].set_index('date').plot()

plt.ylim(100,100000000)
ax.set_yscale('log')

# Plotly Library 

In [ ]:
import plotly.graph_objects as go

In [ ]:
import plotly
plotly.__version__

In [ ]:
country_list = ['India', 'US', 'Italy', 'France', 'Vietnam']

In [ ]:
for each in country_list:
    exp = wp.loc[wp["Country Name"]==each, "2018"]
    

print(exp.dtype)
print(exp)

In [ ]:

for each in country_list:
    var = np.array(wp.loc[wp["Country Name"]==each, "2018"])
    var = var.astype(int)
    df_plot[each] = df_plot[each]/var[0]*100



In [ ]:
fig = go.Figure()

for each in country_list:
    fig.add_trace(go.Scatter(x = df_plot.date, 
                         y = df_plot[each], 
                        mode = 'markers+lines',
                        opacity = 0.9, 
                        line_width = 1,
                        marker_size = 2,
                        name = each))

fig.update_layout(width =900,
                 height = 600, 
                 xaxis_title = 'Time',
                 yaxis_title = "Confirmed Infected People as percentage of total pupulation (source: John Hopkins case, linear-scale)", 
                 )

fig.update_yaxes(type = 'linear')#, range = [0, 100])


# Choose the browser according to your preference

#fig.show(renderer='chrome')
#fig.show(renderer='firefox')

fig.update_layout(xaxis_rangeslider_visible = True)
fig.show()

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    
    html.Label('Multi-Select Country'),
    
    dcc.Dropdown(
        id='country_drop_down',
        options=[
            {'label': 'India', 'value': 'India'},
            {'label': 'US', 'value': 'US'},
            {'label': 'Italy', 'value': 'Italy'},
            {'label': 'France', 'value': 'France'},
            {'label': 'Vietnam', 'value': 'Vietnam'}
        ],
        value=['US', 'India'], # which are pre-selected
        multi=True
    ),   
        
    dcc.Graph(figure=fig, id='main_window_slope')
])


In [ ]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    
    traces = [] 
    for each in country_list:
        traces.append(dict(x=df_plot.date,
                                y=df_plot[each],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=1,
                                marker_size=2, 
                                name=each
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                xaxis_title="Time",
                yaxis_title="Confirmed infected people (source johns hopkins csse, log-scale)",
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        
                      },
                yaxis={'type':"log",
                       'range':'[1.1,5.5]'
                      }
        )
    }

In [ ]:
 app.run_server(debug = True, use_reloader = False)